### Import:

In [2]:
from random import shuffle

import numpy as np
import pandas as pd
from keras.datasets import mnist

import tensorflow as tf
from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical

### Tasks:

<ol>
    <li>Попробуйте обучить, нейронную сеть на Keras(рассмотренную на уроке) на датасете MNIST с другими параметрами. 
        Опишите в комментарии к уроку - какой результата вы добились от нейросети? Что помогло вам улучшить ее точность?</li>
    <li>Поработайте с документацией Keras. Попробуйте найти полезные команды Keras неразобранные на уроке.</li>
</ol>

### Experiments:

Проведу ряд эксперименов, посмотрю влияние разных факторов на качество модели, а так же ряд других экспериментов. Далее заголовками идут условные названия экспериментов, которые потом являются ключами в словаре итоговых метрик experiments_results - поэтому названия именно такого плана (чтобы можно было можно было в качестве ключа указать).

В экспериментах по оценке влияния факторов будет изменен только оцениваемый параметр/фактор, либо несколько, если эксперимент предполагает оценку совместного влияния. Все изменения происходят относительно варианта baseline.

Получаем данные и преобразуем в готовый для обучения вид.

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

Вспомогательные методы.

In [107]:
# Обучение модели с заданными параметрами обучения, возвращает метрику качества.
def learn_evaluate(model,
                   data: tuple,  # X_train, X_test, y_train, y_test.
                   optimizer,
                   loss_func,
                   metrics,
                   epochs,
                   batch_size):
    model.compile(
      optimizer=optimizer,
      loss=loss_func,
      metrics=metrics,
    )

    model.fit(
      data[0],
      to_categorical(data[2]),
      epochs=epochs,
      batch_size=batch_size,
    )

    eval_ = model.evaluate(
      data[1],
      to_categorical(data[3])
    )

    return eval_[-1]


# Многократное обучение модели с заданными параметрами, возваращает среднее значение метрики качества и std.
def learn_evaluate_n_times(n_times,
                           model,
                           data: tuple,  # X_train, X_test, y_train, y_test.
                           optimizer,
                           loss_func,
                           metrics,
                           epochs,
                           batch_size):
    result_metrics = []
    for i in range(n_times):
        result = learn_evaluate(model,
                              data,
                              optimizer,
                              loss_func,
                              metrics,
                              epochs,
                              batch_size)

        result_metrics.append(result)
    result_metrics = np.array(result_metrics)
    return {'mean': round(result_metrics.mean(), 4), 'std': round(result_metrics.std(), 4)}


# Обучение модели и получение предикта по тестовой выборке.
def learn_predict(model,
                   data: tuple,  # X_train, X_test, y_train, y_test.
                   optimizer,
                   loss_func,
                   metrics,
                   epochs,
                   batch_size):
    
    model.compile(
        optimizer=optimizer,
        loss=loss_func,
        metrics=metrics,
    )

    model.fit(
        data[0],
        to_categorical(data[2]),
        epochs=epochs,
        batch_size=batch_size,
    )

    eval_ = model.evaluate(
        data[1],
        to_categorical(data[3])
    )
    
    return model.predict(data[1])


# Метрики качества по разным экспериментам.
experiments_results = {}

##### baseline:

Базовая модель.

In [108]:
model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])


experiments_results['baseline'] =\
learn_evaluate_n_times(n_times=5,
               model=model,
               data=(train_images, test_images, train_labels, test_labels),
               optimizer='adam',
               loss_func='categorical_crossentropy',
               metrics=['accuracy'],
               epochs=5,
               batch_size=32)

Epoch 1/5
60000/60000 [==============================] - 5s 80us/step - loss: 0.3592 - accuracy: 0.8916
Epoch 2/5
60000/60000 [==============================] - 4s 72us/step - loss: 0.1860 - accuracy: 0.9431
Epoch 3/5
60000/60000 [==============================] - 4s 74us/step - loss: 0.1418 - accuracy: 0.9567
Epoch 4/5
60000/60000 [==============================] - 4s 73us/step - loss: 0.1195 - accuracy: 0.9627
Epoch 5/5
10000/10000 [==============================] - 1s 74us/step
Epoch 1/5
60000/60000 [==============================] - 5s 81us/step - loss: 0.0940 - accuracy: 0.9703
Epoch 2/5
60000/60000 [==============================] - 4s 74us/step - loss: 0.0870 - accuracy: 0.9728
Epoch 3/5
60000/60000 [==============================] - 4s 73us/step - loss: 0.0792 - accuracy: 0.9746
Epoch 4/5
60000/60000 [==============================] - 4s 72us/step - loss: 0.0716 - accuracy: 0.9771
Epoch 5/5
10000/10000 [==============================] - 1s 75us/step
Epoch 1/5
60000/60000 [=====

In [5]:
experiments_results

{'baseline': {'mean': 0.9713, 'std': 0.0021}}

##### more_layers:

Больше слоев.

In [6]:
model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(64, activation='relu'),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])

experiments_results['more_layers'] =\
learn_evaluate_n_times(n_times=5,
               model=model,
               data=(train_images, test_images, train_labels, test_labels),
               optimizer='adam',
               loss_func='categorical_crossentropy',
               metrics=['accuracy'],
               epochs=5,
               batch_size=32)

Epoch 1/5
60000/60000 [==============================] - 2s 37us/step - loss: 0.3530 - accuracy: 0.8905
Epoch 2/5
60000/60000 [==============================] - 2s 35us/step - loss: 0.1766 - accuracy: 0.9464
Epoch 3/5
60000/60000 [==============================] - 2s 34us/step - loss: 0.1408 - accuracy: 0.9565
Epoch 4/5
60000/60000 [==============================] - 2s 34us/step - loss: 0.1241 - accuracy: 0.9611
Epoch 5/5
10000/10000 [==============================] - 0s 18us/step
Epoch 1/5
60000/60000 [==============================] - 2s 37us/step - loss: 0.0988 - accuracy: 0.9685
Epoch 2/5
60000/60000 [==============================] - 2s 34us/step - loss: 0.0919 - accuracy: 0.9707
Epoch 3/5
60000/60000 [==============================] - 2s 35us/step - loss: 0.0824 - accuracy: 0.9739
Epoch 4/5
60000/60000 [==============================] - 2s 35us/step - loss: 0.0766 - accuracy: 0.9755
Epoch 5/5
10000/10000 [==============================] - 0s 19us/step
Epoch 1/5
60000/60000 [=====

In [7]:
experiments_results

{'baseline': {'mean': 0.9713, 'std': 0.0021},
 'more_layers': {'mean': 0.9678, 'std': 0.0038}}

##### more_neurones_in_layer

Больше нейронов в одном слое.

In [8]:
model = Sequential([
  Dense(128, activation='relu', input_shape=(784,)),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax'),
])

experiments_results['more_neurones_in_layer'] =\
learn_evaluate_n_times(n_times=5,
               model=model,
               data=(train_images, test_images, train_labels, test_labels),
               optimizer='adam',
               loss_func='categorical_crossentropy',
               metrics=['accuracy'],
               epochs=5,
               batch_size=32)

Epoch 1/5
60000/60000 [==============================] - 3s 47us/step - loss: 0.3129 - accuracy: 0.9047
Epoch 2/5
60000/60000 [==============================] - 3s 45us/step - loss: 0.1516 - accuracy: 0.9524
Epoch 3/5
60000/60000 [==============================] - 3s 45us/step - loss: 0.1140 - accuracy: 0.9642
Epoch 4/5
60000/60000 [==============================] - 3s 45us/step - loss: 0.0946 - accuracy: 0.9699
Epoch 5/5
10000/10000 [==============================] - 0s 24us/step
Epoch 1/5
60000/60000 [==============================] - 3s 53us/step - loss: 0.0766 - accuracy: 0.9753
Epoch 2/5
60000/60000 [==============================] - 3s 50us/step - loss: 0.0632 - accuracy: 0.9794
Epoch 3/5
60000/60000 [==============================] - 3s 51us/step - loss: 0.0579 - accuracy: 0.9809
Epoch 4/5
60000/60000 [==============================] - 3s 51us/step - loss: 0.0526 - accuracy: 0.9823
Epoch 5/5
10000/10000 [==============================] - 0s 25us/step
Epoch 1/5
60000/60000 [=====

In [9]:
experiments_results

{'baseline': {'mean': 0.9713, 'std': 0.0021},
 'more_layers': {'mean': 0.9678, 'std': 0.0038},
 'more_neurones_in_layer': {'mean': 0.9728, 'std': 0.0042}}

##### more_neurones_in_layer_and_more_layers

Одновременно больше нейронов в слое и больше слоев.

In [10]:
model = Sequential([
  Dense(128, activation='relu', input_shape=(784,)),
  Dense(128, activation='relu'), 
  Dense(128, activation='relu'),
  Dense(128, activation='relu'),
  Dense(10, activation='softmax'),
])

experiments_results['more_neurones_in_layer_and_more_layers'] =\
learn_evaluate_n_times(n_times=5,
               model=model,
               data=(train_images, test_images, train_labels, test_labels),
               optimizer='adam',
               loss_func='categorical_crossentropy',
               metrics=['accuracy'],
               epochs=5,
               batch_size=32)

Epoch 1/5
60000/60000 [==============================] - 4s 65us/step - loss: 0.3143 - accuracy: 0.9002
Epoch 2/5
60000/60000 [==============================] - 4s 62us/step - loss: 0.1618 - accuracy: 0.9501
Epoch 3/5
60000/60000 [==============================] - 4s 63us/step - loss: 0.1281 - accuracy: 0.9597
Epoch 4/5
60000/60000 [==============================] - 4s 62us/step - loss: 0.1072 - accuracy: 0.9663
Epoch 5/5
10000/10000 [==============================] - 0s 31us/step
Epoch 1/5
60000/60000 [==============================] - 4s 62us/step - loss: 0.0848 - accuracy: 0.9734
Epoch 2/5
60000/60000 [==============================] - 4s 59us/step - loss: 0.0756 - accuracy: 0.9761
Epoch 3/5
60000/60000 [==============================] - 4s 59us/step - loss: 0.0688 - accuracy: 0.9784
Epoch 4/5
60000/60000 [==============================] - 4s 59us/step - loss: 0.0624 - accuracy: 0.9804
Epoch 5/5
10000/10000 [==============================] - 0s 32us/step
Epoch 1/5
60000/60000 [=====

In [11]:
experiments_results

{'baseline': {'mean': 0.9713, 'std': 0.0021},
 'more_layers': {'mean': 0.9678, 'std': 0.0038},
 'more_neurones_in_layer': {'mean': 0.9728, 'std': 0.0042},
 'more_neurones_in_layer_and_more_layers': {'mean': 0.9713, 'std': 0.0043}}

##### sigmoid_inside

Функция активации внутреннего слоя заменена на sigmoid.

In [12]:
model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='sigmoid'),
  Dense(10, activation='softmax'),
])


experiments_results['sigmoid_inside'] =\
learn_evaluate_n_times(n_times=5,
               model=model,
               data=(train_images, test_images, train_labels, test_labels),
               optimizer='adam',
               loss_func='categorical_crossentropy',
               metrics=['accuracy'],
               epochs=5,
               batch_size=32)

Epoch 1/5
60000/60000 [==============================] - 3s 46us/step - loss: 0.4583 - accuracy: 0.8775
Epoch 2/5
60000/60000 [==============================] - 2s 41us/step - loss: 0.2155 - accuracy: 0.9358
Epoch 3/5
60000/60000 [==============================] - 2s 41us/step - loss: 0.1638 - accuracy: 0.9506
Epoch 4/5
60000/60000 [==============================] - 2s 40us/step - loss: 0.1344 - accuracy: 0.9599
Epoch 5/5
10000/10000 [==============================] - 0s 32us/step
Epoch 1/5
60000/60000 [==============================] - 3s 43us/step - loss: 0.1046 - accuracy: 0.9678
Epoch 2/5
60000/60000 [==============================] - 2s 41us/step - loss: 0.0924 - accuracy: 0.9719
Epoch 3/5
60000/60000 [==============================] - 2s 41us/step - loss: 0.0860 - accuracy: 0.9736
Epoch 4/5
60000/60000 [==============================] - 2s 40us/step - loss: 0.0775 - accuracy: 0.9758
Epoch 5/5
10000/10000 [==============================] - 0s 35us/step
Epoch 1/5
60000/60000 [=====

In [13]:
experiments_results

{'baseline': {'mean': 0.9713, 'std': 0.0021},
 'more_layers': {'mean': 0.9678, 'std': 0.0038},
 'more_neurones_in_layer': {'mean': 0.9728, 'std': 0.0042},
 'more_neurones_in_layer_and_more_layers': {'mean': 0.9713, 'std': 0.0043},
 'sigmoid_inside': {'mean': 0.9678, 'std': 0.0059}}

##### more_epochs

Увеличено кол-во эпох обучения.

In [14]:
model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])


experiments_results['more_epochs'] =\
learn_evaluate_n_times(n_times=5,
               model=model,
               data=(train_images, test_images, train_labels, test_labels),
               optimizer='adam',
               loss_func='categorical_crossentropy',
               metrics=['accuracy'],
               epochs=15,
               batch_size=32)

Epoch 1/15
60000/60000 [==============================] - 3s 47us/step - loss: 0.3623 - accuracy: 0.8903
Epoch 2/15
60000/60000 [==============================] - 3s 43us/step - loss: 0.1882 - accuracy: 0.9428
Epoch 3/15
60000/60000 [==============================] - 3s 43us/step - loss: 0.1452 - accuracy: 0.9556
Epoch 4/15
60000/60000 [==============================] - 3s 43us/step - loss: 0.1229 - accuracy: 0.9621
Epoch 5/15
60000/60000 [==============================] - 3s 43us/step - loss: 0.1042 - accuracy: 0.9678
Epoch 6/15
60000/60000 [==============================] - 3s 43us/step - loss: 0.0944 - accuracy: 0.9705
Epoch 7/15
60000/60000 [==============================] - 3s 44us/step - loss: 0.0843 - accuracy: 0.9739
Epoch 8/15
60000/60000 [==============================] - 3s 43us/step - loss: 0.0803 - accuracy: 0.9748
Epoch 9/15
60000/60000 [==============================] - 3s 43us/step - loss: 0.0743 - accuracy: 0.9764
Epoch 10/15
60000/60000 [==============================

In [15]:
experiments_results

{'baseline': {'mean': 0.9713, 'std': 0.0021},
 'more_layers': {'mean': 0.9678, 'std': 0.0038},
 'more_neurones_in_layer': {'mean': 0.9728, 'std': 0.0042},
 'more_neurones_in_layer_and_more_layers': {'mean': 0.9713, 'std': 0.0043},
 'sigmoid_inside': {'mean': 0.9678, 'std': 0.0059},
 'more_epochs': {'mean': 0.9725, 'std': 0.0018}}

##### larger_batch_size

Увеличено значение параметра batch_size метода fit().

In [16]:
model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])


experiments_results['larger_batch_size'] =\
learn_evaluate_n_times(n_times=5,
               model=model,
               data=(train_images, test_images, train_labels, test_labels),
               optimizer='adam',
               loss_func='categorical_crossentropy',
               metrics=['accuracy'],
               epochs=5,
               batch_size=128)

Epoch 1/5
60000/60000 [==============================] - 1s 20us/step - loss: 0.4703 - accuracy: 0.8641
Epoch 2/5
60000/60000 [==============================] - 1s 15us/step - loss: 0.2284 - accuracy: 0.9325
Epoch 3/5
60000/60000 [==============================] - 1s 15us/step - loss: 0.1808 - accuracy: 0.9456
Epoch 4/5
60000/60000 [==============================] - 1s 16us/step - loss: 0.1490 - accuracy: 0.9548
Epoch 5/5
10000/10000 [==============================] - 0s 42us/step
Epoch 1/5
60000/60000 [==============================] - 1s 20us/step - loss: 0.1176 - accuracy: 0.9638
Epoch 2/5
60000/60000 [==============================] - 1s 16us/step - loss: 0.1025 - accuracy: 0.9687
Epoch 3/5
60000/60000 [==============================] - 1s 16us/step - loss: 0.0929 - accuracy: 0.9715
Epoch 4/5
60000/60000 [==============================] - 1s 16us/step - loss: 0.0846 - accuracy: 0.9737
Epoch 5/5
10000/10000 [==============================] - 0s 41us/step
Epoch 1/5
60000/60000 [=====

In [17]:
experiments_results

{'baseline': {'mean': 0.9713, 'std': 0.0021},
 'more_layers': {'mean': 0.9678, 'std': 0.0038},
 'more_neurones_in_layer': {'mean': 0.9728, 'std': 0.0042},
 'more_neurones_in_layer_and_more_layers': {'mean': 0.9713, 'std': 0.0043},
 'sigmoid_inside': {'mean': 0.9678, 'std': 0.0059},
 'more_epochs': {'mean': 0.9725, 'std': 0.0018},
 'larger_batch_size': {'mean': 0.9691, 'std': 0.0033}}

##### less_data

Уменьшен размер обучающей выборки.

In [18]:
model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])


experiments_results['less_data'] =\
learn_evaluate_n_times(n_times=5,
               model=model,
               data=(train_images[:5000], test_images, train_labels[:5000], test_labels),
               optimizer='adam',
               loss_func='categorical_crossentropy',
               metrics=['accuracy'],
               epochs=5,
               batch_size=32)

Epoch 1/5
5000/5000 [==============================] - 0s 98us/step - loss: 0.9079 - accuracy: 0.7426
Epoch 2/5
5000/5000 [==============================] - 0s 48us/step - loss: 0.4002 - accuracy: 0.8804
Epoch 3/5
5000/5000 [==============================] - 0s 49us/step - loss: 0.3287 - accuracy: 0.9028
Epoch 4/5
5000/5000 [==============================] - 0s 47us/step - loss: 0.2914 - accuracy: 0.9178
Epoch 5/5
10000/10000 [==============================] - 0s 47us/step
Epoch 1/5
5000/5000 [==============================] - 1s 101us/step - loss: 0.2403 - accuracy: 0.9304
Epoch 2/5
5000/5000 [==============================] - 0s 49us/step - loss: 0.2168 - accuracy: 0.9368
Epoch 3/5
5000/5000 [==============================] - 0s 50us/step - loss: 0.1962 - accuracy: 0.9404
Epoch 4/5
5000/5000 [==============================] - 0s 50us/step - loss: 0.1703 - accuracy: 0.9498
Epoch 5/5
10000/10000 [==============================] - 0s 46us/step
Epoch 1/5
5000/5000 [======================

In [19]:
experiments_results

{'baseline': {'mean': 0.9713, 'std': 0.0021},
 'more_layers': {'mean': 0.9678, 'std': 0.0038},
 'more_neurones_in_layer': {'mean': 0.9728, 'std': 0.0042},
 'more_neurones_in_layer_and_more_layers': {'mean': 0.9713, 'std': 0.0043},
 'sigmoid_inside': {'mean': 0.9678, 'std': 0.0059},
 'more_epochs': {'mean': 0.9725, 'std': 0.0018},
 'larger_batch_size': {'mean': 0.9691, 'std': 0.0033},
 'less_data': {'mean': 0.9157, 'std': 0.0095}}

### Выводы по экспериментам по оценке влияния факторов:

In [20]:
experiments_results

{'baseline': {'mean': 0.9713, 'std': 0.0021},
 'more_layers': {'mean': 0.9678, 'std': 0.0038},
 'more_neurones_in_layer': {'mean': 0.9728, 'std': 0.0042},
 'more_neurones_in_layer_and_more_layers': {'mean': 0.9713, 'std': 0.0043},
 'sigmoid_inside': {'mean': 0.9678, 'std': 0.0059},
 'more_epochs': {'mean': 0.9725, 'std': 0.0018},
 'larger_batch_size': {'mean': 0.9691, 'std': 0.0033},
 'less_data': {'mean': 0.9157, 'std': 0.0095}}

Как нейросети сложно строить обобщенные зависимости на малом объеме данных так и человеку. Эксперименты прикидочные, делать далеко идущие выводы по ним сложно и необоснованно, но тем не менее.

- Кол-во слоев немного ухудшило качество и дисперсию, видимо, избыточное кол-во слоев приводит к несуществующим обобщениям и незначительному ухудшению качества.

- Кол-во нейронов в слое качество улучшило - возможно, этот фактор, действительно, влияет положительно.

- При одновременном увеличении кол-ва нейронов в слое и кол-ва слоев средняя метрика не изменилась, а разброс вырос, в таком контексте лучше выбрать базовую модель - из-за меньшего разброса и большей простоты модели.

- Сигмоида вместо relu во внутреннем слое ощутимо ухудшила среднюю метрику и ощутимо увеличила дисперсию.

- Увеличение кол-ва эпох сказалось положительно на среднем качестве и позволило добиться наименьшей из всех экспериментов дисперсии результатов.

- Увеличение параметра batch_size метода fit() модели ухудшил качество.

- Ну и ожидаемо значительное уменьшение объема обучающей выборки значительно и уменьшило значение метрики качества. Размер был уменьшен в порядка 10 раз. Эффект на качество - наибольший из всех экспериментов и очень ощутимый. Вероятно, это самый мощный фактор качества обучения, что соответствует общеизвестным фактам. В то же время почти очевидно, что дальнейшее добавление данных будет все меньше влиять на качество, т.е. можно предположить, что в некотором диапазоне приращение объема обучающей выборки будет давать взрывной рост качества, при обучении моделей следует, конечно, стремиться получить объем данных, лежащий за пределами этого диапазона справа.

### Extra experiments:

##### Эксперимент: как данный тип нейросети сможет распознать изображение, переврнутое вниз головой если обучался на не перевернутой выборке.

In [21]:
# Возвращает массив в обратном порядке.
def resort_arr(arr):
    return np.array([arr[-i] for i in range(1, len(arr) + 1)])

In [22]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

test_images_ = np.array([resort_arr(val) for val in test_images])

train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))
test_images_ = test_images_.reshape((-1, 784))

In [23]:
model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])


reflected_results = learn_evaluate_n_times(n_times=5,
               model=model,
               data=(train_images, test_images_, train_labels, test_labels),
               optimizer='adam',
               loss_func='categorical_crossentropy',
               metrics=['accuracy'],
               epochs=5,
               batch_size=32)

reflected_results

Epoch 1/5
60000/60000 [==============================] - 3s 56us/step - loss: 0.3582 - accuracy: 0.8922
Epoch 2/5
60000/60000 [==============================] - 3s 52us/step - loss: 0.1879 - accuracy: 0.9425
Epoch 3/5
60000/60000 [==============================] - 3s 51us/step - loss: 0.1438 - accuracy: 0.9556
Epoch 4/5
60000/60000 [==============================] - 3s 51us/step - loss: 0.1218 - accuracy: 0.9623
Epoch 5/5
10000/10000 [==============================] - 1s 50us/step
Epoch 1/5
60000/60000 [==============================] - 4s 59us/step - loss: 0.0980 - accuracy: 0.9688
Epoch 2/5
60000/60000 [==============================] - 3s 54us/step - loss: 0.0844 - accuracy: 0.9734
Epoch 3/5
60000/60000 [==============================] - 3s 54us/step - loss: 0.0797 - accuracy: 0.9746
Epoch 4/5
60000/60000 [==============================] - 3s 54us/step - loss: 0.0733 - accuracy: 0.9765
Epoch 5/5
10000/10000 [==============================] - 1s 51us/step
Epoch 1/5
60000/60000 [=====

{'mean': 0.332, 'std': 0.0214}

In [37]:
model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])

preds_ = learn_predict(model=model,
               data=(train_images, test_images_, train_labels, test_labels),
               optimizer='adam',
               loss_func='categorical_crossentropy',
               metrics=['accuracy'],
               epochs=1,
               batch_size=32)

Epoch 1/1
10000/10000 [==============================] - 1s 57us/step


In [38]:
preds = np.argmax(preds_, axis=1)

preds

array([3, 5, 1, ..., 9, 3, 9], dtype=int64)

In [39]:
preds = pd.DataFrame({'true': test_labels, 'preds': preds})
preds['match'] = 0
preds.loc[preds['true'] == preds['preds'], 'match'] = 1
preds.head()

,true,preds,match
0,7,3,0
1,2,5,0
2,1,1,1
3,0,0,1
4,4,7,0


Среднее качество предиктов в разрезе лейблов.

In [40]:
preds.groupby('true')['match'].mean()

true
0    0.659184
1    0.643172
2    0.040698
3    0.753465
4    0.402240
5    0.022422
6    0.000000
7    0.111868
8    0.375770
9    0.171457
Name: match, dtype: float64

Наиболее частый ответ модели в разрезе true значений.

In [43]:
preds.groupby('true')['preds'].apply(lambda x: x.mode().iloc[0])

true
0    0
1    1
2    3
3    3
4    9
5    2
6    9
7    2
8    3
9    4
Name: preds, dtype: int64

##### Вывод по эксперименту с переворачиванием картинки вверх ногами:

При переворачивании картинок из тестового набора вверх ногами качество упало с 0,97 до 0,33. Очевидно, что модель не умеет распозонавать перевернутые изображения. Так же довольно очевидно, что в перевернутых цифрах тем не менее можно находить те же самые черты, что и в неперевернутых аналогах, об этом говорит качество 0,33, которое больше 0,1, которое соответсвует рандому.

Построил в разрезе отдельных цифр среднее качество, а так же наиболее частый предикт модели в на каждой true цифре, т.е. на какую цифру какой результат выдавал модель чаще всего.

Результаты довольно интересные и в то же время вполне предсказуемые в большинстве своем.

На симметричные относительно горизонтали цифры (относительно конечно потому что там имеет место наклон) модель после переворачивания выдала наилучшие результаты: 0, 1 (в наборе почти все единички как палочки без зазубрины сверху), 3, 8. А вот по таким цифрам как 2,5,6,9 модель очень сильно ошибалась, как можно видеть из таблицы с модами - она принимала их за их перевернутые условные братья близнецы - 6 принимала за 9, 7 за 2. Но это прослеживаемые закономерности, в целом видимая картина шире этих закономерностей.

##### Эксперимент: как нейросеть сможет обучаться и каково будет качество её предсказаний если входные данные кодировать по некоторой заданной маске.

In [44]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = (train_images / 255) - 0.5
test_images = (test_images / 255) - 0.5

train_images = train_images.reshape((-1, 784))
test_images = test_images.reshape((-1, 784))

Готовлю маску для кодирования изображения (пиксели при упощении 2D массива меняются местами рандомно, но всегда по одному алгоритму).

In [102]:
mask = {}

arr = [i for i in range(784)]
arr_ = arr.copy()
shuffle(arr)

for i in range(len(arr)):
    mask[arr[i]] = arr_[i]

def mask_image(arr_2d, mask):
    result_arr = [0 for i in range(len(arr_2d))]
    for i_new, i_old in mask.items():
        result_arr[i_old] = arr_2d[i_new]
    return np.array(result_arr)

In [99]:
train_images_ = np.array([mask_image(val, mask) for val in train_images])
test_images_ = np.array([mask_image(val, mask) for val in test_images])

In [100]:
model = Sequential([
  Dense(64, activation='relu', input_shape=(784,)),
  Dense(64, activation='relu'),
  Dense(10, activation='softmax'),
])


experiments_result =\
learn_evaluate_n_times(n_times=5,
               model=model,
               data=(train_images_, test_images_, train_labels, test_labels),
               optimizer='adam',
               loss_func='categorical_crossentropy',
               metrics=['accuracy'],
               epochs=5,
               batch_size=32)

Epoch 1/5
60000/60000 [==============================] - 4s 75us/step - loss: 0.3517 - accuracy: 0.8961
Epoch 2/5
60000/60000 [==============================] - 4s 68us/step - loss: 0.1849 - accuracy: 0.9438
Epoch 3/5
60000/60000 [==============================] - 4s 69us/step - loss: 0.1458 - accuracy: 0.9549
Epoch 4/5
60000/60000 [==============================] - 4s 69us/step - loss: 0.1207 - accuracy: 0.9620
Epoch 5/5
10000/10000 [==============================] - 1s 68us/step
Epoch 1/5
60000/60000 [==============================] - 5s 78us/step - loss: 0.0954 - accuracy: 0.9702
Epoch 2/5
60000/60000 [==============================] - 4s 71us/step - loss: 0.0862 - accuracy: 0.9722
Epoch 3/5
60000/60000 [==============================] - 4s 71us/step - loss: 0.0787 - accuracy: 0.9749
Epoch 4/5
60000/60000 [==============================] - 4s 70us/step - loss: 0.0700 - accuracy: 0.9776
Epoch 5/5
10000/10000 [==============================] - 1s 69us/step
Epoch 1/5
60000/60000 [=====

##### Вывод по эксперименту с запограммированным перемешиванием.

Запрограммированное перемешивание пикселей, т.е. любой алгоритм перемешивания пикселей в изображении (главное чтобы он был одинаковым для всех изображений) никак не влияет на качество обучения модели. С одной стороны предсказуемо, с другой стороны ни разу не помогает в проникновении внутрь черного ящика нейросети и понимании того, как именно нейросеть классифицирует, учится и узнает классы. Получается, геометрическое расположение пикселей относительно друг друга (близость и т.д.) никак не влияет на возможность и сам процесс узнавания моделью.

Нда, похоже, мои выкрутасы с прогонами обучения по 5 раз на модели делают не совсем то, что я ожидал. Похоже не 5 моделей с нуля обучаются, а одна модель дообучается 5 раз. Ну, в целом сильно на рассуждения это не влияет.

### Интересные функциональные возможности и команды Keras из документации:

Давно мне было интересно - когда читаешь все эти статьи про мега-нейросети, которые обучались месяц - неужели они поставили на месяц и ушли "пить чай", а если ошибка, а если баг, а если надо как-то замерить промежуточные результаты и что-то скорретировать. Похоже, коллбеки - та функциональность, которая позволяет разруливать все эти вопросы и процессы.

Только почему-то пример из документации падает с ошибкой.

In [ ]:
# Define the Keras model to add callbacks to
def get_model():
    model = keras.Sequential()
    model.add(keras.layers.Dense(1, input_dim=784))
    model.compile(
        optimizer=keras.optimizers.RMSprop(learning_rate=0.1),
        loss="mean_squared_error",
        metrics=["mean_absolute_error"],
    )
    return model

# Load example MNIST data and pre-process it
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 784).astype("float32") / 255.0
x_test = x_test.reshape(-1, 784).astype("float32") / 255.0

# Limit the data to 1000 samples
x_train = x_train[:1000]
y_train = y_train[:1000]
x_test = x_test[:1000]
y_test = y_test[:1000]

class CustomCallback(keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        keys = list(logs.keys())
        print("Starting training; got log keys: {}".format(keys))

    def on_train_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop training; got log keys: {}".format(keys))

    def on_epoch_begin(self, epoch, logs=None):
        keys = list(logs.keys())
        print("Start epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        print("End epoch {} of training; got log keys: {}".format(epoch, keys))

    def on_test_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start testing; got log keys: {}".format(keys))

    def on_test_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop testing; got log keys: {}".format(keys))

    def on_predict_begin(self, logs=None):
        keys = list(logs.keys())
        print("Start predicting; got log keys: {}".format(keys))

    def on_predict_end(self, logs=None):
        keys = list(logs.keys())
        print("Stop predicting; got log keys: {}".format(keys))

    def on_train_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: start of batch {}; got log keys: {}".format(batch, keys))

    def on_train_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Training: end of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: start of batch {}; got log keys: {}".format(batch, keys))

    def on_test_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Evaluating: end of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_begin(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: start of batch {}; got log keys: {}".format(batch, keys))

    def on_predict_batch_end(self, batch, logs=None):
        keys = list(logs.keys())
        print("...Predicting: end of batch {}; got log keys: {}".format(batch, keys))
        
model = get_model()
model.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=1,
    verbose=0,
    validation_split=0.5,
    callbacks=[CustomCallback()],
)

res = model.evaluate(
    x_test, y_test, batch_size=128, verbose=0, callbacks=[CustomCallback()]
)

res = model.predict(x_test, batch_size=128, callbacks=[CustomCallback()])